In [ ]:
import csv
import networkx as nx 
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns

# Punto 0 - Carga de la red

In [ ]:
def build_graph():
    with open('World.csv') as flights:
        # skippeo el primer elemento (origen, destino)
        reader = iter(csv.reader(flights, delimiter=','))
        next(reader)
        return nx.from_edgelist(list(map(lambda x: (x[0], x[1]), reader)))

G = build_graph()

# Punto 1 - Basic stats (Diametro/Grado promedio/Clustering)

In [ ]:
def basic_stats(G: nx.Graph, log=False, name=None):
    stats = {
        'Nodos': len(G.nodes), 
        'Aristas': len(G.edges), 
        'Diametro': nx.diameter(G),
        'Grado promedio': np.mean(np.array(list(map(lambda t: t[1], list(G.degree))))),
        'Clustering': np.mean(np.array(list(nx.clustering(G).values()))),
        'Puentes globales': list(nx.bridges(G)),
        'Puentes locales': list(nx.local_bridges(G, with_span=False))
    }
    if log:
        from pprint import pprint
        print(f"\n|======Stats para: {name}======|\n")
        print(f"Cantidad de paises: {stats['Nodos']}\n")
        print(f"Cantidad de vuelos: {stats['Aristas']}\n")
        print(f"Diametro: {stats['Diametro']}\n")
        print(f"Grado promedio: {stats['Grado promedio']}\n")
        print(f"Coeficiente de clustering promedio: {stats['Clustering']}\n")
        pprint(f"Puentes globales: {stats['Puentes globales']}")
        print()
        pprint(f"Puentes locales: {stats['Puentes locales']}")
        print()
    return stats

stats = basic_stats(G, True, "Red de Vuelos")

# Punto 2 - Homofilia

Decidi analizar si hay homofilia por cruce de continente (un par de aristas cruzan continente si los paises de sus vertices se encuentran en continentes distintos). Para esto utilizo la libreria pycountry y pycountry_convert

In [ ]:
import pycountry_convert as pcc

""" Disclaimer: Tomado de https://github.com/mbuchwald/social-networks-utils/blob/main/homofilia.py"""
def contar_aristas(grafo):
    contador = 0
    for v in grafo:
        contador += len(list(grafo.neighbors(v)))
    return contador if nx.is_directed(grafo) else contador // 2


def proporcion_cruzan_campo(grafo, mapper=None):
    """
    :param grafo:
    :param mapper: un mapper de vertice a un tipo (por defecto, grafo[v])
    :return: devuelve la proporcion real que se obtiene de cruces respecto del total de aristas
    """
    aristas_totales = contar_aristas(grafo)
    cruzan_bloque = 0
    visitados = set()
    mapper = mapper if mapper is not None else (lambda k: grafo[k]["type"])

    for v in grafo:
        for w in grafo.neighbors(v):
            if not nx.is_directed(grafo) and w in visitados:
                continue
            if mapper(v) != mapper(w):
                cruzan_bloque += 1
        visitados.add(v)
    return cruzan_bloque / aristas_totales


def proporcion_por_tipo(grafo, mapper=None):
    """
    :param grafo:
    :param mapper: un mapper de vertice a un tipo (por defecto, grafo[v])
    :return: la proporcion que hay de cada tipo
    """
    mapper = mapper if mapper is not None else (lambda k: grafo[k]["type"])
    cantidades = {}
    for v in grafo:
        cantidades[mapper(v)] = cantidades.get(mapper(v), 0) + 1
    props = {}
    for c in cantidades:
        props[c] = cantidades[c] / len(grafo)
    return props


""" End disclaimer """

def homophily_by_country(G: nx.Graph):
    print("\n|======HOMOPHILY======|\n")
    def set_country_attributes(G: nx.Graph):
        continents = {
            'NA': 'North America',
            'SA': 'South America', 
            'AS': 'Asia',
            'OC': 'Australia',
            'AF': 'Africa',
            'EU': 'Europe'
        }

        # A riesgo de quedar como un bruto, estos son los paises que aparecen en el grafo pero no estan en el listado 
        # pongo sus continentes ya que simplemente es buscar en google
        not_found_countries = {
            'Netherlands Antilles': 'Europe', 
            "Cote d'Ivoire": 'Africa', 
            'Congo (Brazzaville)': 'Africa', 
            'Congo (Kinshasa)': 'Africa', 
            'Saint Helena': 'Africa', 
            'Reunion': '', 
            'Western Sahara': 'Africa', 
            'Virgin Islands': 'North America', 
            'Burma': 'Asia', 
            'East Timor': 'Asia',
        } 

        count = 0
        for country in list(G.nodes):
            try:
                if country == 'Reunion':
                    G.nodes[country]["continent"] = 'Africa'
                    continue
                alpha_code = pcc.country_name_to_country_alpha2(country, cn_name_format="default")
                continent_code = pcc.country_alpha2_to_continent_code(alpha_code)
                if continent_code == '':
                    print(country)
                G.nodes[country]["continent"] = continents[continent_code]
                count += 1
            except BaseException as e:
                G.nodes[country]["continent"] = not_found_countries[country]

        crossing_proportion = proporcion_cruzan_campo(G, lambda x: G.nodes[x]["continent"])
        props = proporcion_por_tipo(G, lambda x: G.nodes[x]["continent"])
        probability_crossing = 0
        for p in props:
            probability_crossing += props[p] * (1 - props[p])

        # retorna la proporcion de aristas que cruzan de continente y el threshold a partir del cual no habría 
        # homofilia por continente
        return crossing_proportion, probability_crossing, props

    crossing_continent_proportion, threshold, props = set_country_attributes(G)
    if crossing_continent_proportion < threshold:
        print(f"Hay homofilia por cruce de continente, siendo el threshold {threshold}"
               f" y la proporcion de aristas que cruzan {crossing_continent_proportion}")
    else:
        print("No hay homofilia por continente")
    print(props)

homophily_by_country(G)

In [ ]:
from itertools import product


external = 0
local = 0
for c in ["Europe", "Africa", "North America", "South America", "Asia", "Australia"]:
    countries = list(filter(lambda x: G.nodes[x]["continent"] == c, list(G.nodes)))
    print(f"{c} tiene {len(countries)} paises")
    country_pairs = list(product(countries, countries))

    continent_local = 0
    continent_external = 0
    for x in countries:
        neighbors = list(nx.neighbors(G, x))
        for y in neighbors:
            if y not in countries and G.has_edge(x, y):
                external += 1
                continent_external += 1 
            if y in countries and G.has_edge(x, y):
                local += 1
                continent_local += 1 
    print(c, continent_local//2, continent_external)

print(f"Vuelos locales al continente: {local//2}, Vuelos intercontinentales: {external//2}")

# Punto 3 - Puentes (globales/locales)

Ya cubierto en el punto 1 - los puentes globales son:

- ('Fiji', 'Tuvalu'),
- ('United States', 'American Samoa'),
- ('United Kingdom', 'Saint Helena'),
- ('Canada', 'Saint Pierre and Miquelon'),
- ('Antigua and Barbuda', 'Montserrat'),
- ('New Zealand', 'Niue'),
- ('South Africa', 'Lesotho'),
- ('South Africa', 'Swaziland'),
- ('Burma', 'Myanmar')

A estos, se les suman 2 puentes locales (todos los puentes globales tambien son locales por definicion):

- ('Papua New Guinea', 'Micronesia'),
- ('Micronesia', 'Marshall Islands'),

# Punto 4 - Centralidad

Se calcula en gephi ya que es extremadamente sencillo graficar y calcular las centralidades.

Elijo calcular **Betweeness centrality** (de ahora en mas BC). Se podria pensar que un camino minimo en esta red nos devuelve, para un par de paises, el vuelo con menos escalas entre ellos. Ojo, esto no significa en primer lugar que sea la forma más rápida de llegar de un país a otro, ni tampoco que sea el unico camino minimo entre ellos, ni que sea una escala que se haga realmente a nivel global. Los paises con mayor puntaje de centralidad seran aquellos intermediarios que tienen más salida al resto de los paises del mundo. Podemos pensar por un lado que los paises muy grandes a nivel adquisitivo, como Estados Unidos, Reino unido, entre otros, tendran una tendencia a mayor puntaje de BC, ya que es probable que tengan una mayor infraestructura aeroportuaria que les permita llegar a más paises que aquellos con menor infraestructura.

Los paises con mas vuelos a mayor cantidad de paises del resto del mundo (aquellos con mayor grado en la red), van a tener una gran tendencia a ser parte de escalas para pares de paises que no tengan vuelos directos entre ellos, y por ende van a tener inflado su puntaje de BC.
![Alt_text](Betweenness_centrality.png)

En la imagen se muestra como por ejemplo Estados Unidos (primer lugar) y Francia (segundo lugar) son los que mayor puntaje de BC tienen, siendo estos por ende los paises que mas participan de caminos minimos entre otros pares de paises, y en nuestra interpretacion, los que mas figuran en ciertas escalas.

La siguiente celda devuelve todos los caminos minimos que incluyen a Francia o a Estados unidos como intermediario de ese camino minimo.

In [ ]:
import functools
from pprint import pprint

france_shortest_paths = [x for k, v in nx.shortest_path(G).items() for x in v.values() if 'France' in x and len(x) > 2 and x[0] != 'France' and x[len(x) - 1] != 'France']
print('===== FRANCE ======')
count_map = {}

def add_count(x):
    if (x[0], x[1]) and (x[1], x[0]) not in count_map:
        count_map[(x[0], x[1])] = 1
    else:
        if (x[0], x[1]) in count_map:
            count_map[(x[0], x[1])] += 1
        elif (x[1], x[0]) in count_map:
            count_map[(x[1], x[0])] += 1

list(map(add_count, map(lambda x: [G.nodes[x[0]]["continent"], G.nodes[x[len(x) - 1]]["continent"]], france_shortest_paths)))

count_map_france = dict(sorted(count_map.items(), key=lambda item: item[1]))

print(count_map_france)

""" que es esta locura de Africa? """
# pprint.pprint(list(filter(lambda x: G.nodes[x[0]]["continent"] == 'Africa' and G.nodes[x[len(x)-1]]["continent"] == 'Africa', france_shortest_paths)))

usa_shortest_paths = [x for k, v in nx.shortest_path(G).items() for x in v.values() if 'United States' in x and len(x) > 2 and x[0] != 'United States' and x[len(x) - 1] != 'United States']
count_map = {}
list(map(add_count, map(lambda x: [G.nodes[x[0]]["continent"], G.nodes[x[len(x) - 1]]["continent"]], usa_shortest_paths)))
count_map_usa = dict(sorted(count_map.items(), key=lambda item: item[1]))
print('====== USA ======')
print(count_map_usa)
pprint(usa_shortest_paths)

# Punto 5 - Modelados (Erdos-Renyi / Preferential Attachment)

In [ ]:
def plot_density_graph(G: nx.Graph, logarithmic = False):
    degree_sequence = sorted((d for _, d in G.degree()), reverse=True)
    g = sns.displot(degree_sequence, kind="kde", log_scale=(logarithmic, logarithmic),)
    g.fig.set_size_inches(7, 7)
    plt.xlabel('Grado')
    plt.xlim(left=0)


In [ ]:
plot_density_graph(G, logarithmic=False)
plt.title('Densidad de grado - Red de vuelos')
plt.savefig('g.jpg', dpi=600)
plot_density_graph(G, logarithmic=True)
plt.title('Densidad de grado - Red de vuelos (log-log)')
plt.savefig('g_log_log.jpg', dpi=600)

## Erdos-Renyi

Para este modelo, utilizamos como parámetros la cantidad de nodos en la red y el grado promedio. El grado promedio lo necesitamos para estimar la probabilidad de que un par de aristas cualesquiera esten conectadas entre si (p = k/(n - 1)).

In [ ]:
# utilizo el modulo de modelos!

""" Disclaimer: Tomado de https://github.com/mbuchwald/social-networks-utils/blob/main/modelos.py """
def erdos_renyi(cant, k):
    import random
    valores = list(range(cant))
    g = nx.Graph()
    g.add_nodes_from(valores)
    p = k / (cant - 1)
    for v in range(cant):
        for w in range(v + 1, cant):
            if v == w: continue
            if random.uniform(0, 1) < p:
                g.add_edge(v, w)
    return g

""" End disclaimer """

def build_erdos_renyi_model(n_size, k):
    return erdos_renyi(n_size, k)

G_erdos_renyi = build_erdos_renyi_model(stats['Nodos'], stats['Grado promedio'])

In [ ]:
# estadisticas basicas sobre este modelo
basic_stats(G_erdos_renyi, log=True)

In [ ]:
plot_density_graph(G_erdos_renyi, logarithmic=False)
plt.title('Densidad de grado - Modelo Erdös-Rényi')
plt.savefig('g_erdos_renyi.jpg', dpi=600)
plot_density_graph(G_erdos_renyi, logarithmic=True)

## Preferential attachment

Para este modelo, necesitamos obtener la distribucion de grados de la red original, para poder calcular el alpha por estimador de maxima verosimiltud al tratar de ajustar esta distribucion a una ley de potencias (sin cutoff). Una vez obtenido el alpha, podemos ajustar un modelo de preferential attachment.

Hay varios hiperparametros para tweakear. Por la distribucion de grados, asumí que la ley de potencias se empieza a cumplir a partir de x = 10 (la cantidad de nodos con grado mayor a 10 empiezan a disminuir siguiendo una power law, a ojo).

La cantidad de nodos es un parametro fijo (229), pero no asi el valor de k, un estimado de cuantas nuevas aristas se podrían llegar a agregar a la red a medida que nuevos nodos se van uniendo. Puse un valor que no es el grado promedio (k = 10), pero que, junto con alpha, me dio una cantidad de aristas muy similar a la de la red original.

In [ ]:
import random
""" Disclaimer: Tomado de https://github.com/mbuchwald/social-networks-utils/blob/main/modelos.py """
def elegir_preferntial(grafo, banned):
    grados_entrada = [0] * len(grafo)
    total = 0
    for v in range(len(grafo)):
        for w in grafo.neighbors(v):
            if w in banned:
                continue
            grados_entrada[w] += 1
            total += 1
    if total == 0:
        return None

    aleat = random.uniform(0, total)
    sumando = 0
    for i in range(len(grafo)):
        sumando += grados_entrada[i]
        if sumando > aleat:
            return i


def preferential_attachment(dirigido, alfa, cant, k):
    p = 1 - (1/(alfa - 1))
    valores = list(range(cant))
    g = nx.DiGraph() if dirigido else nx.Graph()
    g.add_nodes_from(valores)
    for v in range(cant):
        banned = set([v])
        for i in range(int(k)):
            preferential = random.uniform(0, 1) < p
            ya_agregado = False
            if preferential and v > 0:
                w = elegir_preferntial(g, banned)
                if w is not None:
                    banned.add(w)
                    g.add_edge(v, w)
                    ya_agregado = False
            if not ya_agregado:
                w = random.choice(list(set(range(cant)) - set([v])))
                g.add_edge(v, w)

    return g


""" End Disclaimer """
def mle_power_law(samples, xmin):
    return 1 + len(samples) * (math.pow(sum(np.log(x/xmin) if x >= xmin else 0 for x in samples), -1))

def build_preferential_attachment_model(G, xmin):
    degree_sequence = sorted((d for _, d in G.degree()), reverse=True)
    alpha = mle_power_law(degree_sequence, xmin = xmin)
    print(f"Alpha: {alpha}")
    return preferential_attachment(False, alpha, len(list(G.nodes)), k=10)


G_preferential_attachment = build_preferential_attachment_model(G, 10)
basic_stats(G_preferential_attachment)

In [ ]:
plot_density_graph(G_preferential_attachment, logarithmic=False)
plt.title("Densidad de grados - Preferential Attachment")
plt.savefig('g_preferential.jpg', dpi=600)
plot_density_graph(G_preferential_attachment, logarithmic=True)

# Anonymous Walks

Obtenemos una representacion de la red por anonymous walks, luego para las dos simuladas. El tamaño del walk lo vamos variando para obtener embeddings de mayor dimension y luego de haber calculado los 3 embeddings para cada red, loggeamos las dos distancias coseno que se obtienen. 

Para disminuir el tiempo de ejecucion, se paraleliza utilizando un thread pool.

La realidad es ambos modelos se ajustan de una manera muy similar (similaridad por distancia coseno), ya que las distancias calculadas son muy similares entre si.

- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 2 es: 0.0
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 2 es: 0.0
 
- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 3 es: 6.77975163740907e-05
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 3 es: 1.9191540288066022e-05

- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 4 es: 0.00010649682631547197
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 4 es: 0.00013434615719309928

- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 5 es: 0.0001934262895926242
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 5 es: 0.0002066068468873361

- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 6 es: 0.000339797855971824
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 6 es: 0.00030387081425864437

- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 7 es: 0.0005029675404226719
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 7 es: 0.0004554220572019485
 
- La distancia coseno entre la red y Erdos-Renyi, con tamaño de camino 8 es: 0.0006236696712117462
- La distancia coseno entre la red y Preferential Attachment, con tamaño de camino 8 es: 0.0005945686316904952

In [ ]:
import multiprocessing as mp
from itertools import repeat

""" Disclaimer: Tomado de https://github.com/mbuchwald/social-networks-utils/blob/main/metricas.py """
def distancia_coseno(a, b):
    from numpy import linalg as LA
    return 1 - np.inner(a, b) / (LA.norm(a) * LA.norm(b))
""" End disclaimer """


""" Disclaimer: Tomado de https://github.com/mbuchwald/social-networks-utils/blob/main/embeddings.py """
from math import log, e, ceil
import numpy as np
import random
from numpy import linalg as LA

DIFFERENT_WALKS = { # incluye bucles
    2: 2,
    3: 5,
    4: 15,
    5: 52,
    6: 203,
    7: 877,
    8: 4140,
    9: 21147,
    10: 115975,
    11: 678570,
    12: 4213597
}


def ln(num):
    return log(num, e)


def _cant_annonymous_walks(length, error=0.1, delta=0.01):
    nu = DIFFERENT_WALKS[length]
    return ceil((ln(2**nu - 2) - ln(delta)) * (2 / (error**2)))


def _camino_a_clave(camino):
    return "-".join(map(lambda v: str(v), camino))


def _annon_enum_rec(pasos_restantes, mapeo, camino=[], vs_en_camino=0, admite_bucles=False):
    if pasos_restantes == 0:
        mapeo[_camino_a_clave(camino)] = len(mapeo)
        return
    nuevo = vs_en_camino + 1
    ultimo = camino[-1] if len(camino) > 0 else None
    for i in range(1, nuevo + 1):
        if i == ultimo and not admite_bucles:
            continue
        camino.append(i)
        vs_en_este_camino = vs_en_camino + (1 if i == nuevo else 0)
        _annon_enum_rec(pasos_restantes - 1, mapeo, camino, vs_en_este_camino)
        camino.pop()


def _enumerar_anonymous_walks(length):
    mapeo = {}
    _annon_enum_rec(length, mapeo)
    return mapeo


def _random_walk(grafo, length):
    v = random.choice(list(grafo.nodes))
    camino = [v]
    while len(camino) < length:
        v = random.choice(list(grafo.neighbors(v)))
        camino.append(v)
    return camino


def _anonymize_walk(camino):
    translate = {}
    camino_trans = []
    for v in camino:
        if v not in translate:
            translate[v] = len(translate) + 1
        camino_trans.append(translate[v])
    return camino_trans


def anoymous_walks(grafo, length):
    '''
    :param grafo: Grafo a calcularle el embedding por anonymous_walk
    :param length:
    :return:
    '''
    cantidad = _cant_annonymous_walks(length)
    mapeo = _enumerar_anonymous_walks(length)
    contadores = [0] * len(mapeo)
    for i in range(cantidad):
        camino = _random_walk(grafo, length)
        contadores[mapeo[_camino_a_clave(_anonymize_walk(camino))]] += 1

    vector = np.array(contadores)
    return list(mapeo.keys()), vector / LA.norm(vector)

""" End disclaimer """


def cosine_distance_anonymous_walk(other_networks, walk_size):
    print("")
    _, og_network_emb = anoymous_walks(G, walk_size)
    for name, net in other_networks.items():
        _, emb_g = anoymous_walks(net, walk_size)
        print(f"La distancia coseno entre la red y {name}, con tamaño de camino {walk_size} es: {distancia_coseno(og_network_emb, emb_g)}")
    

other_netwoks = {'Erdos-Renyi': G_erdos_renyi, 'Preferential Attachment': G_preferential_attachment}
walk_sizes = [2,3,4,5,6,7,8]
with mp.Pool(mp.cpu_count()) as pool:
    pool.starmap(cosine_distance_anonymous_walk, zip(repeat(other_netwoks), walk_sizes))